In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import warnings   
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)

In [3]:
data = pd.read_excel('data_140000_200000.xlsx')
data1 = pd.read_excel('data_4000_8500.xlsx')

In [4]:
data0 = data.drop('id', axis = 1)
data1 = data1.drop('Индекс (id)', axis = 1)
data0.columns = ['Наименование', 'Код_ЕП_РФ_подкатегория', 'Подкатегория_текст']
data1.columns = ['Наименование', 'Код_ЕП_РФ_подкатегория', 'Подкатегория_текст']

df = pd.concat([data0, data1])

In [5]:
df = df.reset_index(drop = True)
df = df.drop_duplicates(keep = 'first')

In [6]:
df.head()

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у..."
1,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...
4,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...


In [7]:
df.nunique()

Наименование              160583
Код_ЕП_РФ_подкатегория       666
Подкатегория_текст           645
dtype: int64

In [8]:
df['Код_ЕП_РФ_подкатегория'].value_counts()

9300.1                    86752
2364.1                     6622
5990.1                     3163
5970.1                     2932
2293.1                     2743
                          ...  
9441.4; 9436.1                1
2293.2; 2293.5; 2293.1        1
3456.11                       1
9295.1; 9295                  1
1483.1; 2293                  1
Name: Код_ЕП_РФ_подкатегория, Length: 666, dtype: int64

В данных есть колоноки с несколькими категориями

In [20]:
df['multiple_codes'] = df['Код_ЕП_РФ_подкатегория'].apply(lambda x : 1 if ';' in x else 0)

In [21]:
df['multiple_codes'].value_counts()

0    162226
1      1023
Name: multiple_codes, dtype: int64

In [22]:
df[df['multiple_codes'] == 0].nunique()

Наименование              159709
Код_ЕП_РФ_подкатегория       403
Подкатегория_текст           385
multiple_codes                 1
Код_подкатегория             385
dtype: int64

In [23]:
df[df['multiple_codes'] == 1].nunique()

Наименование              973
Код_ЕП_РФ_подкатегория    263
Подкатегория_текст        260
multiple_codes              1
Код_подкатегория          260
dtype: int64

Drop invalid rows

In [24]:
df = df[df['multiple_codes'] == 0].drop('multiple_codes', axis=1)
df.head()

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у...",261
1,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,11
2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,11
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...,442
4,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...,51


Закодируем для удобства предсказания

In [25]:
le = LabelEncoder()
df['Код_подкатегория'] = le.fit_transform(df['Подкатегория_текст'])

In [26]:
df.nunique()

Наименование              159709
Код_ЕП_РФ_подкатегория       403
Подкатегория_текст           385
Код_подкатегория             385
dtype: int64

In [27]:
df.head()

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у...",156
1,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,7
2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,7
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...,267
4,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...,35


In [28]:
df.isna().sum()

Наименование              0
Код_ЕП_РФ_подкатегория    0
Подкатегория_текст        0
Код_подкатегория          0
dtype: int64

Почему кол-во кодов подкатегорий не равно ков-ву описаний?

In [29]:
df

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у...",156
1,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,7
2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,7
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...,267
4,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...,35
...,...,...,...,...
280643,"Посуда чугунная: кастрюли, сковороды, котлы, к...",9695.2,Посуда чугунная черная,246
280647,Посуда из полипропилена и полистирола одноразо...,2293.5,"Посуда, в том числе одноразового применения (к...",247
280674,"Посуда из керамики для взрослых: чашки, пиалы ...",5990.1,"Посуда керамическая (фарфоровая, полуфарфорова...",238
280684,Посуда из пластмассы для взрослых: фляги для в...,2293.5,"Посуда, в том числе одноразового применения (к...",247


In [30]:
from tqdm import tqdm

In [31]:
codes = {}
counter = 0
for i in tqdm(range(len(df))):
    #print(codes)
    if df.iloc[i, 2] in codes:
        if codes[df.iloc[i, 2]] == df.iloc[i, 1]:
            continue 
        else:
            print(f'{codes[df.iloc[i, 2]]} != {df.iloc[i, 1]}, {i}')
            counter += 1
    else:
        codes[df.iloc[i, 2]] = df.iloc[i, 1]
    

  4%|██▌                                                                      | 5775/162226 [00:00<00:13, 11842.45it/s]

9392.1 != 2386.3, 3660


  6%|████▏                                                                    | 9374/162226 [00:00<00:12, 11845.17it/s]

9392.1 != 2386.3, 7764
9296.23 != 9291.2, 8160
9296.23 != 9291.2, 8401
9296.23 != 9291.2, 8612
9296.23 != 9291.2, 9414
9296.23 != 9291.2, 9818
9398.1 != 9391.9, 9849
5463.9 != 8460.1, 9870


  7%|█████▏                                                                  | 11710/162226 [00:01<00:13, 10994.61it/s]

...


9392.1 != 2386.3, 155688
9392.1 != 2386.3, 156373
9392.1 != 2386.3, 156375
9296.23 != 9291.2, 156510
9392.1 != 2386.3, 156539
9392.1 != 2386.3, 156674
9392.1 != 2386.3, 156712
9392.1 != 2386.3, 156824
9392.1 != 2386.3, 156844
9392.1 != 2386.3, 156899
2180.1 != 2387.2, 157022
2180.1 != 2387.2, 157023
3183.26 != 3451.11, 157190
9296.23 != 9291.2, 157477
9296.23 != 9291.2, 157523
9392.1 != 2386.3, 157597

 99%|██████████████████████████████████████████████████████████████████████▏| 160331/162226 [00:15<00:00, 10609.86it/s]


9296.23 != 9291.2, 157677
9296.23 != 9291.2, 157818
9392.1 != 2386.3, 157926
9392.1 != 2386.3, 157946
9392.1 != 2386.3, 157988
9392.1 != 2386.3, 158011
3414.16 != 3414.18, 158051
9296.23 != 9291.2, 158161
9296.23 != 9291.2, 158163
3414.16 != 3414.18, 158231
2180.1 != 2387.2, 158369
9392.1 != 2386.3, 158657
9392.1 != 2386.3, 158797
9392.1 != 2386.3, 158951
9296.23 != 9291.2, 159075
9296.23 != 9291.2, 159154
9296.23 != 9291.2, 159156
9296.23 != 9291.2, 159196
9392.1 != 2386.3, 159377
2553.1 != 2557.1, 159381
9392.1 != 2386.3, 159471
3414.16 != 3414.18, 159619


100%|███████████████████████████████████████████████████████████████████████| 162226/162226 [00:15<00:00, 10645.88it/s]

9296.23 != 9291.2, 160451
2553.1 != 2557.1, 160460
9296.23 != 9291.2, 160470
9392.1 != 2386.3, 160525
2553.1 != 2557.1, 160580
2440.1 != 2440, 160598
9296.23 != 9291.2, 160718
9392.1 != 2386.3, 160871
9296.23 != 9291.2, 160893


In [43]:
df[(df.Подкатегория_текст == 'Средства дезинфицирующие') & (df.Код_ЕП_РФ_подкатегория != '2386.3')]

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
3798,Дезинфицирующее средство с моющим эффектом,9392.1,Средства дезинфицирующие,318
5489,Дезинфицирующее средство с моющим эффектом «Mi...,9392.1,Средства дезинфицирующие,318
5496,Дезинфицирующее средство,9392.1,Средства дезинфицирующие,318
5503,Дезинфицирующее средство (салфетки),9392.1,Средства дезинфицирующие,318
9900,Средство дезинфицирующее «Клерсайд Спорицидал ...,9392.1,Средства дезинфицирующие,318
...,...,...,...,...
267539,"Средство дезинфицирующее с моющим эффектом ""Же...",9392.1,Средства дезинфицирующие,318
267582,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318
267583,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318
267603,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318


In [46]:
df[(df.Подкатегория_текст == 'Средства дезинфицирующие') & (df.Код_ЕП_РФ_подкатегория == '2386.3')]

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
6978,Средство дезинфицирующее (кожный антисептик),2386.3,Средства дезинфицирующие,318
18582,"Средство зоогигиеническое «Ребавагин», выпуска...",2386.3,Средства дезинфицирующие,318
24463,"Средство для предварительной, окончательной и ...",2386.3,Средства дезинфицирующие,318
24576,"Средство дезинфицирующее ""DGM Steriguard EASY ...",2386.3,Средства дезинфицирующие,318
26012,"Средство дезинфицирующее ""DGM Steriguard EASY ...",2386.3,Средства дезинфицирующие,318
...,...,...,...,...
264756,"Средство дезинфицирующее ""ДЕЗАВИД для Сауны""",2386.3,Средства дезинфицирующие,318
265382,"Средство инсектицидное ""Жидкость от комаров дл...",2386.3,Средства дезинфицирующие,318
265527,"Средство родентицидное ""BROS гранулы от мышей ...",2386.3,Средства дезинфицирующие,318
267074,Средства дезинфекционные,2386.3,Средства дезинфицирующие,318


In [44]:
df[(df.Подкатегория_текст == 'Средства дезинфицирующие')]

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
3798,Дезинфицирующее средство с моющим эффектом,9392.1,Средства дезинфицирующие,318
5489,Дезинфицирующее средство с моющим эффектом «Mi...,9392.1,Средства дезинфицирующие,318
5496,Дезинфицирующее средство,9392.1,Средства дезинфицирующие,318
5503,Дезинфицирующее средство (салфетки),9392.1,Средства дезинфицирующие,318
6978,Средство дезинфицирующее (кожный антисептик),2386.3,Средства дезинфицирующие,318
...,...,...,...,...
267582,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318
267583,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318
267598,Средство дезинфицирующее «Перекись водорода 6%»,2386.3,Средства дезинфицирующие,318
267603,Средство дезинфицирующее «DGM Steriguard EASY ...,9392.1,Средства дезинфицирующие,318


In [183]:
counter

904

In [32]:
codes_all = {}
counter = 0
for i in tqdm(range(len(df))):
    #print(codes)
    if df.iloc[i, 2] in codes_all:
        codes_all[df.iloc[i, 2]].append(df.iloc[i, 1])
    else:
        codes_all[df.iloc[i, 2]] = [df.iloc[i, 1]]

100%|███████████████████████████████████████████████████████████████████████| 162226/162226 [00:13<00:00, 11665.90it/s]


In [33]:
#codes_all

In [34]:
import statistics
from statistics import mode
 
def most_common(List):
    return(mode(List))

In [35]:
codes_matching = {}
for i in tqdm(codes_all.keys()):
    codes_matching[i] =  most_common(codes_all[i])

100%|█████████████████████████████████████████████████████████████████████████████| 385/385 [00:00<00:00, 27524.49it/s]


In [36]:
codes_df = pd.DataFrame(codes_matching, index = [0]).T.reset_index()
codes_df.columns = ['text', 'code']
codes_df.head()

,text,code
0,"Лекарственные средства, зарегистрированные в у...",9300.1
1,Аккумуляторы и аккумуляторные батареи никель-м...,3482.2
2,Приборы и аппараты для электролечения высокоча...,9444.4
3,Блоки оконные и балконные дверные из алюминиев...,5270.1
4,Изделия из бумаги бытового и санитарно-гигиени...,5463.1


In [51]:
num_invalid_codes = {}
for key in codes_matching.keys():
    num_invalid_codes[key] = 0
        

In [52]:
for i in tqdm(range(len(df))):
    if df.iloc[i, 1] != codes_matching[df.iloc[i, 2]]:
        num_invalid_codes[df.iloc[i, 2]] += 1

100%|███████████████████████████████████████████████████████████████████████| 162226/162226 [00:09<00:00, 16717.86it/s]


In [54]:
num_invalid_codes1 = {}
for i in num_invalid_codes.keys():
    if num_invalid_codes[i] != 0:
        num_invalid_codes1[i] = num_invalid_codes[i]

In [55]:
num_invalid_codes1

{'Среды питательные микробиологические, основы питательные и сырье биологическое для вирусологических питательных сред, применяемых в медицине': 3,
 'Материалы стоматологические': 36,
 'Средства дезинфицирующие': 158,
 'Удобрения минеральные': 129,
 'Комплектные трансформаторные подстанции': 15,
 'Камеры сборные одностороннего обслуживания': 9,
 'Средства защиты растений химические (пестициды)': 7,
 'Рукава резиновые напорные с текстильным каркасом': 26,
 'Продукция микробиологической промышленности': 9,
 'Изделия для внутреннего протезирования': 93,
 'Белково-витаминно-минеральные и амидо-витаминно-минеральные концентраты': 190,
 'Линзы для коррекции зрения': 9,
 'Инструменты оттесняющие': 1,
 'Материалы хирургические, средства перевязочные': 22,
 'Инструменты режущие и ударные с острой (режущей) кромкой': 2,
 'Стеклоочистители': 2}

In [200]:
#codes_df.to_csv('codes.csv', index = False)

Save data

In [147]:
df.to_csv('data.csv', index = False)

In [185]:
df.to_pickle('data.pkl')

Посмотрим на ошибки в кодах еще раз

In [213]:
codes = pd.DataFrame(df.Код_подкатегория.value_counts())
codes = codes[codes.Код_подкатегория < 3]
inds = codes.index

In [214]:
codes = codes.reset_index()
codes.columns = ['code', 'count']
codes

,code,count
0,285,2
1,297,2
2,357,2
3,171,2
4,362,2
5,381,2
6,366,2
7,3,2
8,16,2
9,42,2


In [218]:
df[df.Код_подкатегория.isin(inds)]

,Наименование,Код_ЕП_РФ_подкатегория,Подкатегория_текст,Код_подкатегория
30307,Выключатели автоматические вакуумные,3414.7,Выключатели автоматические быстродействующие д...,49
33127,Пигменты цветные порошковые (краска разметочна...,2322.1,Ультрамарины для красок для розничной торговли...,360
36348,Иммуноглобулины диагностические эшерихиозные т...,9381.5,Гамма- и иммуноглобулины гетерогенные,52
43516,Инвентарь для прикладных видов спорта: Луки сп...,9616.2,Холодное оружие: луки спортивные массовые,0
51890,"Покрышка для велосипедов, торговых марок: KEND...",2526,"Велошины, велопокрышки, велокамеры и велоизделия",42
65275,сыворотки иммунные,9382.4,Сыворотки прочие,339
67473,Щебень из плотных горных пород (граниты) для с...,5711,Материалы строительные нерудные,171
67948,РЕАКТОР ШУНТИРУЮЩИЙ,3414.19,Реакторы и реакторное оборудование для электро...,298
68073,Щебень из плотных горных пород (граниты) для с...,5711,Материалы строительные нерудные,171
70817,Ветеринарный препарат торговой марки FARMA PET...,9387,Аллергены (включая аллергены для ветеринарии),14
